<a href="https://colab.research.google.com/github/highpix/depixy/blob/master/depixy.ipynb" target="_parent"><img src="https://i.ibb.co/c8Br2S1/logoza-ru.png" alt="Open In Colab"/></a>

<b><font color="black" size="+4">Лицевой Депикселизатор</font></b>

Лецивой депикселизатор находиться на стадии beta pre-release если вы имеете датапаки с лицами людей пожайлуста отправьте их в мой Telegarm!

<b><font color="black" size="+2">...</font></b>

Основано на [TenserFlow](https://www.tensorflow.org/) и [Pulse](https://github.com/adamian98/pulse)

Алгоритм на стадии обучения! Возможны ошибки и баги.

Автора можно поддержать положив немного битка на кошелек - depixy0dc7ZJADp9DJUVUT7d4bvHB2NDPgN

Telegram: [@nyanway](https://t.me/nyanway)

Скачать алгоритм можно из репозитория по ссылке -> [Скачать](https://github.com/adamian98/pulse)

---
##### <font color='red'>В настоящее время Google Drive используется в качестве хранилища весов модели, поэтому из-за постоянных загрузок этих файлов при выполнении следующего блока может быть получено сообщение об ошибке «Google Drive Quota Exceeded» (Квота на Google Drive превышена) или «No such file or directory: '/content/pulse/runs/face.png'». Если вы столкнулись с этой ошибкой, повторите попытку позже в тот же день или вернитесь завтра.</font>

```
Чтобы начать, нажми на кнопку (куда указывает красная стрелка), после чего дождись завершения выполнения блока.
```



In [ ]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Начинаем!</font></b>
#@markdown **После запуска этого блока тебе нужно прокрутить страницу вниз, а затем загрузить пиксельную квадратную фотографию, в которую целиком помещается вся человеческая голова. Depixy лучше всего работает на изображениях, в которых люди смотрят прямо в камеру. Пример:**

#@markdown ![example](https://github.com/tg-bomze/Face-Depixelizer/raw/master/example.jpg)

#@markdown *Обрезать фото ты можешь [ТУТ](https://www.iloveimg.com/crop-image)*

#@markdown ---
import torch
import torchvision
from pathlib import Path
if not Path("PULSE.py").exists():
  if Path("pulse").exists():
    %cd /content/pulse
  else:
    !git clone https://github.com/adamian98/pulse
    %cd /content/pulse
    !mkdir input/
    toPIL = torchvision.transforms.ToPILImage()
    toTensor = torchvision.transforms.ToTensor()
    from bicubic import BicubicDownSample
    D = BicubicDownSample(factor=1)

import os
from io import BytesIO
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
from PULSE import PULSE
from google.colab import files
from bicubic import BicubicDownSample
from IPython import display
from IPython.display import display
from IPython.display import clear_output
import numpy as np
from drive import open_url
from mpl_toolkits.axes_grid1 import ImageGrid
%matplotlib inline

#@markdown ## Базовые настройки:
#@markdown ##### *Если ты уже загрузил фотографию и просто хочешь поэкспериментировать с настройками, то убери галочку ниже*:
upload_new_photo = True #@param {type:"boolean"}


if upload_new_photo == True:
  !rm -rf /content/pulse/input/face.png
  clear_output()
  uploaded = files.upload()
  for fn in uploaded.keys():
    print('Ты загрузил файл "{name}" размером {length} байт'.format(
        name=fn, length=len(uploaded[fn])))
  os.rename(fn, fn.replace(" ", ""))
  fn = fn.replace(" ", "")

  if(len(uploaded.keys())!=1): raise Exception("Тебе необходимо загрузить только одно изображение.")

  face = Image.open(fn)
  face = face.resize((1024, 1024), Image.ANTIALIAS)
  face = face.convert('RGB')
  face_name = 'face.png'
  face.save(face_name)
  %cp $face_name /content/pulse/input/

  images = []
  imagesHR = []
  imagesHR.append(face)
  face = toPIL(D(toTensor(face).unsqueeze(0).cuda()).cpu().detach().clamp(0,1)[0])
  images.append(face)

#@markdown ---
#@markdown ## Расширенные настройки:
#@markdown ##### *Если ты хочешь получить более точный результат, измени следующие переменные, оптимально настроенные по умолчанию*:

input_dir = '/content/pulse/input/'
output_dir = '/content/pulse/runs/'
seed = 100 #@param {type:"integer"}
epsilon = 0.02 #@param {type:"slider", min:0.01, max:0.03, step:0.01}
noise_type = 'trainable'  #@param ['zero', 'fixed', 'trainable']
optimizer = 'adam'  #@param ['sgd', 'adam','sgdm', 'adamax']
learning_rate = 0.4 #@param {type:"slider", min:0, max:1, step:0.05}
learning_rate_schedule = 'linear1cycledrop'  #@param ['fixed', 'linear1cycle', 'linear1cycledrop']
steps = 100 #@param {type:"slider", min:100, max:2000, step:50}
clear_output()

seed = abs(seed)
print('Примерное время генерации: {} сек.\n'.format(round(0.23*steps)+6))
!python run.py \
  -input_dir $input_dir \
  -output_dir $output_dir \
  -seed $seed \
  -noise_type $noise_type \
  -opt_name $optimizer \
  -learning_rate $learning_rate \
  -steps $steps \
  -eps $epsilon \
  -lr_schedule $learning_rate_schedule

#@markdown ---
#@markdown *Если во время выполнения произошла ошибка или кнопка "**Обзор**" не активна, то попробуй повторно запустить этот блок*

fig, (ax1, ax2) = plt.subplots(1, 2)
ax1.imshow(mpimg.imread('/content/pulse/input/face.png'))
ax1.set_title('Оригинал')
ax2.imshow(mpimg.imread('/content/pulse/runs/face.png'))
ax2.set_title('Результат')
plt.show()

In [ ]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Скачать итоговое изображение</font></b>
try: files.download('/content/pulse/runs/face.png')
except: raise Exception("No result image")